In [1]:
import geopandas as gpd
import pandas as pd
import rasterio
from rasterstats import zonal_stats
import matplotlib.pyplot as plt
from rasterio.plot import show
import os

In [2]:
os.chdir('c:\\Users\\Jesse\\OneDrive\\Documenten\\Master BAOR\\Thesis\\GitHub\\dicra\\analytics\\sandbox\\geospatial_internship\\datasets')
fire_data = pd.read_csv('telangana_fires.csv')
telangana_shape = gpd.read_file('telangana_shapefile.geojson')

os.chdir('c:\\Users\\Jesse\\OneDrive\\Documenten\\Master BAOR\\Thesis\\GitHub\\dicra\\src\\data_preprocessing\\tsdm\\')
district_boundaries = gpd.read_file('District_Boundary.shp')

os.chdir('c:\\Users\\Jesse\\OneDrive\\Documenten\\Master BAOR\\Thesis\\GitHub\\dicra\\analytics\\sandbox\\notebooks\\crop_fires\\Classification_Fires')
fires_2021 = gpd.read_file('telanganafire.geojson')

In [3]:
fire_data['acq_date'] = pd.to_datetime(fire_data['acq_date'])
fires_2021['acq_date'] = pd.to_datetime(fires_2021['acq_date'])

In [37]:
#Create geodataframe from the data
geo_fire_data = gpd.GeoDataFrame(fire_data,geometry = gpd.points_from_xy(fire_data.longitude,fire_data.latitude), crs = {'init': 'epsg:4326'}) 

#Make sure the
geo_fire_data['geometry'] = geo_fire_data['geometry'].geometry.to_crs(epsg = 4326)
district_boundaries['geometry'] = district_boundaries['geometry'].geometry.to_crs(epsg = 4326)

#Combine fires from 2021 with the rest
geo_fire_data = geo_fire_data.append(fires_2021)
geo_fire_data['fireID'] = [a for a in range(0, len(geo_fire_data))]

polygons = []

#-------create a buffer of square of 1km size using buffer function from shapely-----#

for i in geo_fire_data.geometry:
    p1 = i
    buffer = p1.buffer(0.004504505, cap_style = 3)         #500m = 0.004504505 and cap_style 3 is square box of same of 1km side length
    polygons.append(buffer)

#create a new column in GeoDataFrame newdf and dump polygon buffer of respective point values
geo_fire_data['geometry buffered'] = polygons 

geo_fire_data['acq_date'] =  pd.to_datetime(geo_fire_data['acq_date'])
geo_fire_data['year'] = (geo_fire_data['acq_date']).dt.year

geo_fire_data = geo_fire_data[geo_fire_data['year']<2022]
geo_fire_data

os.chdir('c:\\Users\\Jesse\\OneDrive\\Documenten\\Master BAOR\\Thesis\\GitHub\\dicra\\analytics\\sandbox\\notebooks\\crop_fires\\Exploration\\Landuse_Analysis')
df = pd.DataFrame(columns=geo_fire_data.columns)

for a in geo_fire_data['year'].unique():
    year = a
    year_data = geo_fire_data[geo_fire_data['year'] == year]
    if (year==2015) | (year == 2016):
        tiff = '01-01-2017.tif'
    else:
        tiff = '01-01-'+str(year)+'.tif'
    lulc = rasterio.open(tiff, mode = 'r')
    lulc_array = lulc.read(1) # landuse corresponding to each rasterpixel, so we extracted the pixel values from the raster
    # affine: 1: corresponds to the width of each pixel, 2: row rotation, 3: x-coordinate of the upper left pixel, 4: column rotation, 5: height of each pixel, 6: y-coordinate of the upper left pixel
    affine = lulc.transform
    cmap = {1: 'Water', 2: 'Trees', 4: 'Flooded Vegetation', 5: 'Crops', 7: 'Built Area', 8: 'Bare Ground', 9: 'Snow/Ice', 10: 'Clouds', 11: 'Rangeland'}
    year_data['Land use'] = 0
    year_data['Trees Around'] = 0

    for i in range(0, len(year_data)):
        test = zonal_stats(year_data['geometry'].iloc[i], lulc_array, affine = affine, geojson_out = True, stats = 'majority', nodata = lulc.nodata, categorical=True, category_map = cmap, all_touched=True)
        try: 
            year_data['Land use'].iloc[i] = test[0]['properties']['majority']
        except:
            year_data['Land use'].iloc[i] = 'unknown'

    for i in range(0, len(year_data)):
        test = zonal_stats(year_data['geometry buffered'].iloc[i], lulc_array, affine = affine, geojson_out = True, stats = 'majority', nodata = lulc.nodata, categorical=True, category_map = cmap, all_touched=True)

        if 'Trees' in list(test[0]['properties'].keys()):
            year_data['Trees Around'].iloc[i]=1
        else:
            year_data['Trees Around'].iloc[i]=0

    
    df = pd.concat([df, year_data], axis = 0)

    

c:\Users\Jesse\Anaconda3\envs\gurobi\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
c:\Users\Jesse\Anaconda3\envs\gurobi\lib\site-packages\geopandas\geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
c:\Users\Jesse\Anaconda3\envs\gurobi\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See th

In [38]:
df['agricultural'] = 0
df['agricultural strict'] = 0

for i in range(0, len(df)):
    if (df['Land use'].iloc[i] == 5) | (df['Land use'].iloc[i] == 4) :
        df['agricultural'].iloc[i] = 1
    if ((df['Land use'].iloc[i] == 5) | (df['Land use'].iloc[i] == 4)) & (df['Trees Around'].iloc[i]==0):
        df['agricultural strict'].iloc[i] = 1


c:\Users\Jesse\Anaconda3\envs\gurobi\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [41]:
os.chdir('c:\\Users\\Jesse\\OneDrive\\Documenten\\Master BAOR\\Thesis\\GitHub\\dicra\\analytics\\sandbox\\notebooks\\crop_fires\\Classification_Fires')

In [42]:
df[['fireID', 'latitude', 'longitude', 'brightness', 'scan', 'track',
       'acq_date', 'acq_time', 'satellite', 'instrument', 'confidence',
       'version', 'bright_t31', 'frp', 'daynight', 'type', 'geometry','agricultural', 'agricultural strict']].to_csv('fires_data_classified.csv')

In [43]:
df = gpd.GeoDataFrame(df,geometry = df.geometry, crs = {'init': 'epsg:4326'}) 

c:\Users\Jesse\Anaconda3\envs\gurobi\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [44]:
df[['fireID', 'latitude', 'longitude', 'brightness', 'scan', 'track',
       'acq_date', 'acq_time', 'satellite', 'instrument', 'confidence',
       'version', 'bright_t31', 'frp', 'daynight', 'type', 'geometry','agricultural', 'agricultural strict']].to_file('fires_data_classified.geojson', driver="GeoJSON") 

# Classification (Using Shivangs Approach)

###Reference : https://github.com/luckyw0w/Data4Policy/tree/main/Geospatial%20Data%20Science%20Internship/ShivangPandey

So, the buffer function of geopandas (based on the shapely library) is to create a buffer polygon around a point. The first parameter of the buffer is to tell length in the degree to mark the point in a square pixel and 'cap_style' to tell the type of polygon. here 'cap_style = 3' means square polygon for buffer point. 

We need to create a polygon, because MODIS Thermal Anomalies & Fire Daily data is calculated on 1km resolution and given esri LULC data is at 10m resolution and we can say it like that 1 pixel of MODIS fire data should be as big as 10000 pixels of LULC map.
How we are going to use that? by creating a polygon as a size of 10000 pixels of LULC map.


###Reference: https://www.arcgis.com/home/item.html?id=d6642f8a4f6d4685a24ae2dc0c73d4ac

In [97]:
os.chdir('c:\\Users\\Jesse\\OneDrive\\Documenten\\Master BAOR\\Thesis\\GitHub\\dicra\\analytics\\notebooks\\crop_fires')
tiff = 'vmcx_mosiac_reprj.tif'

data = geo_fire_data
data['index'] = data.index

#------------------Checking probability in mosaic tiff file----------------------#
import time                                      #to calculate time taken to run the model
start_time = time.time()                         #start time of the program

index_list = []                               #declaring empty to get corresponding id of fire point
flag_list = []                                #empty list to store class of corresponding fire points based polygon
    
for j in range(len(data)):                #iterating all rows of dataframe to get point info

    stats = zonal_stats(data.iloc[j].geometry, tiff, stats="*", categorical=True)         #getting statistics from the raster point 
    i = stats[0]                                                                          #storing statsistical dictionary in a value
    if i['count'] != 0:                                                                   #check if polygon is within the tiff file or not
        index = data.iloc[j]['index']                                                     #get id of polygon
        index_list.append(index)                                                          #store id in a list
        if (5 in i) or (4 in i): 
            #print(i)                                                         #check if crop class or flooded vegitation is in polygon region or not
            flag = 1                                                                      #Mark whether occurence is in crop field
        else:
            flag = 0                                                                      # if crop class is not in polygon, return 0 pixels
        flag_list.append(flag)                                                            #store class value in a list
        
print("--- %s seconds ---" % (time.time() - start_time))                                  #print total time taken to run code 

#dictionary created wth fireID and class (1,0)
id_class = {key: value for key, value in zip(index_list, flag_list)}

#creating tuple of each key and  value pair
data_items = id_class.items()                                                
#dumping all tuples in a list
data_list = list(data_items)

#creating DataFrame with id and class values
class_df = pd.DataFrame(data_list, columns= ['fireID','class (1,0)'])


c:\Users\Jesse\Anaconda3\envs\gurobi\lib\site-packages\rasterstats\io.py:301: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")


--- 167.75042867660522 seconds ---


In [98]:
geo_fire_data['agricultural'] = class_df['class (1,0)']

In [99]:
geo_fire_data[['fireID', 'latitude', 'longitude', 'brightness', 'scan', 'track',
       'acq_date', 'acq_time', 'satellite', 'instrument', 'confidence',
       'version', 'bright_t31', 'frp', 'daynight', 'type', 'geometry','agricultural']].to_csv('fires_data_classified.csv')